In [1]:
import cv2          #Libreria de vision artificial
import os           #Trabajar con informacion del sistema operativo
import numpy as np  #Operaciones matriciales 

In [2]:
def extraer_fotogramas(video):                  #Funcion creada para extraer cada fotograma y guardarlos
                                                #y guardarlos en una lista
    ruta_actual   = os.getcwd()                     #Cargamos la ruta donde se encuentra el proyecto
    fotogramas    = []                              #Lista dedicada a contener los fotogramas

    vidcap        = cv2.VideoCapture(ruta_actual+'/'+video) #Creacion de objeto video capture
    success,image = vidcap.read()                           #Se lee el primer fotograma
    count         = 0                                       #Se establece un contador

    while success:                                          #El Objeto video capture controla la extraccion 
        fotogramas.append(image)                            #Se guarda el frame
        success,image = vidcap.read()                       #Se pregunta por los siguientes 
        count        += 1                                   #Se aumenta el contador

    return fotogramas

fotogramas = extraer_fotogramas('time_laps3.mp4')
print('el total de fotogramas es:', len(fotogramas)-1)

el total de fotogramas es: 216


In [3]:
#En esta celda se encuentra la fucnion tomada de: https://www.learnopencv.com/image-alignment-feature-based-…/
#Ingresan dos imagenes en formato "NDARRAY" el fotograma a alinear y el fotograma de referencia
#Retorna dos "NDARRAY" una con la imagen alineada y los match que hizo para alinearla
MAX_FEATURES = 5000000
GOOD_MATCH_PERCENT = 0.5
 
def alignImages(im1, im2):
    
    # Convert images to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    #cv2.imwrite("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg,imMatches

In [4]:
def alinear(fotogramas):
    
    alineados = []                                              #Lista para guardar alineaciones
    matches = []                                                #Lista para guardar matches
    referencia = fotogramas[-1]                                 #Se toma el ultimo cuadro como referencia

    for fotograma in fotogramas:                                #Se iteran todos los fotogramas
        alineado, match = alignImages(fotograma, referencia)    #Se hace la alineacion
        matches.append(match)                                   #Se guardan los matches
        alineados.append(alineado)                              #Se guarda alineados
    
    return alineados                                            #Se retornan los fotogramas alineados

alineados = alinear(fotogramas)

In [5]:
def juntar_cuadros(fotogramas, alineados):
    
    dim = np.shape(fotogramas[0])                              #Se guardan las dimensiones individuales de cada fotograma
    unidos = []                                                #Lista para guardar los cuadros concatenados
    
    for i, fotograma in enumerate(fotogramas):                 #Se iteran los fotogramas

        fotograma1 = alineados[i]                              
        unidos.append(np.concatenate((fotograma,fotograma1)))  #Se concatenana los fotogramas originales con los alineados y se guardan
        
    return unidos

unidos = juntar_cuadros(fotogramas, alineados)

In [6]:
def armar_video(unidos):
    
    fourcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')                 #Formato del video
    dim1 = np.shape(unidos[0])                                          #Dimensiones de espaciales del video
    out = cv2.VideoWriter('matches.mp4',fourcc, 15, (dim1[1], dim1[0])) #Se crea el objeto para escribir el video

    for match in unidos:                                                #Se iteran los cuadros
        out.write(match)                                                #Se guardan
    out.release()                                                       #Se libera el objeto
    
armar_video(unidos)